<img src="https://comp116sp19.cs.unc.edu/images/COMP116Logo.png" style="display:inline; width:200px" />

# Assignment 4 Estimating Statistical Significance


This NASA post, <a href="http://www.nasa.gov/topics/earth/features/midweek_rainfall.html">NASA Data Link Pollution to Rainy Summer Days in the Southeast</a>, claims that in the southeastern US it rains more Tuesday through Thursday than it does Saturday through Monday. The presence of a seven-day cycle in the weather is "eerie" evidence that human activity influences the weather.

Your mission in this assignment is to see if you can validate their claim using data from the instruments at RDU airport. To quote from the article:

>Rainfall measurements collected from ground-based gauges can vary from one gauge site to the next because of fickle weather patterns. So, to identify any kind of significant weekly rainfall trend, Bell and colleagues looked at the big picture from Earth's orbit.

We are not really equipped to confirm or refute their claim but we're in the southeast, and have some data, let's see what we can do.

I have collected 10 years of data into a single file, `krdu-2001-2010.csv`, in a format suitable for use with np.loadtxt(). These data have 4 columns; year, month, day, and rainfall in inches.

First, you should read these posts by Allen Downey:

1. <a href="http://allendowney.blogspot.com/2011/05/there-is-only-one-test.html">There is only one test!</a>
2. <a href="http://allendowney.blogspot.com/2011/06/more-hypotheses-less-trivia.html">More hypotheses, less trivia.</a> Pay special attention to the paragraph <b>Permutation</b> under <b>Difference in means</b>. You shouldn't use his code; write your own.

## Learning objectives

The goal of this assignment is to give you more experience writing functions with loops and to expose you to an important use of computers in science.

## What to do

1. Read the data.
2. Determine the days of the week from the dates. (You can't just slice the array)
3. Write a function to get the average daily rainfall during midweek (Tuesday through Thursday) and weekend (Thursday through Monday).  Treat Friday as neither a weekday nor weekend.
4. Report the average rainfall for midweek and weekend, and their difference (delta). I provide a check for this so you can be sure you're on the right track.
5. Determine and report the p value (the likelihood that the effect is not real) by simulation. You'll need to first, compute the delta which in our case is the difference between the means for midweek and weekend. Then you'll run the function many times, each time permuting the day labels, and counting the number of times that the difference between the new means is greater than delta. Now divide that count by the number of trials you ran. That will be the p value. If count is 0, that means you didn't find even a single permutation that produced a greater difference in means; the effect is very likely real. On the other hand, if count is huge, then there were many permutations that produced greater differences, so the difference is likely just random.


# Notes

## Datetime tip



**1)** The <a href="https://docs.python.org/3.5/library/datetime.html#datetime.date.weekday"><b>weekday</b></a> method of the <b>date</b> class from the <b>datetime</b> module will be useful for getting the day of the week from the date. **Note:** The `weekday()` method returns `0` for Monday, `1` for Tuesday, `2` for Wednesday, etc until `6` for Sunday.

The date class knows nothing about numpy so you'll need to use a loop to process all the days. Also your data will be floats but the date function insists on ints; you'll need to use the int() function to convert them to int. 


You use it like this:


In [1]:
from datetime import date

# later in your code when you want to determine the day of the week
# you create a date object from a year, month, and day.
# I'll fake up a year, month, and day so we can see it work
year = 2000
month = 1
day = 1
do = date(year, month, day)
# and use its weekday method
day1 = do.weekday()

# or do it all in one step
day2 = date(year, month, day).weekday()

print(do, day1, day2)

2000-01-01 5 5


I would highly recommend that you copy the above cell and play with datetime.
For example, what day of the week were you born on?
When is the next Friday the 13th?

## Random permutations of numbers

**2)** The <a href="http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.permutation.html"> `np.random.permutation`</a> function will be useful for permuting the data.
For example, let's assume you had an NumPy array of 10 numbers, 0, 1, 2, ... 9.
But you want them in some random order.

You use it like this:

In [2]:
# first I'll create some array I want to shuffle.
import numpy as np
np.random.seed(10) # Only used so we all get the same result in class

A = np.arange(10)
B = np.random.permutation(A)
B

array([8, 2, 5, 6, 3, 1, 0, 7, 4, 9])

## Time required for submission

**3)** You'll need to use a loop to run your simulation many times. I found that 1000 trials gives a faily stable result and doesn't require too long to run (good for debugging). Feel free to play with larger numbers but **I will time limit your submissions and assume they failed if they require more than 30 seconds to run on my computer**. 

You can use the time package to determine the time your program took to execute.

In [3]:
import time

# Get the start time
t0 = time.time()

# Now sleep for 5 seconds
length = 5
time.sleep( length )

# Now check how long it took
print('A sleep(', length, ') took', time.time() - t0, 'seconds!')

A sleep( 5 ) took 5.000861167907715 seconds!


## Checks

Since I'm not writing a lot of code for you, I can't give you the usual super structured checks.  I'm going to give you some very general ones to help keep you on track. You'll be responsible for making sure you put your various results into the correct variable names.

You should *play* with the data.
For example, you can add to every weekday's rain amount and see what happens.
There are checks in here, but you should make sure your program works by playing with the data.

**4)** I confirmed my intuition about how this should work by tweaking the data. For example, if I add 0.1 inches of rain to every Tuesday, the p value drops to zero (very significant) but if I replace the rainfall with random numbers it rises to near 0.5 (not significant at all).

## Setup

Set up the environment for the assignment.

In [4]:
%matplotlib inline
import numpy as np
import pylab
import time
from datetime import date
import comp116
check, report = comp116.start('A4')

In [5]:
# fill in your onyen (not PID) here
Author = 'mdubois6'
Collaborators = []

In [6]:
check('0.1: Author filled in', Author != 'youronyen', points=1)
check('0.2: Collaborators filled in', Collaborators != ['list', 'their', 'onyens', 'here'], points=1)

0.1: Author filled in appears correct
0.2: Collaborators filled in appears correct


## Read the data

First you should read the data. The delimiter is comma.

In [7]:

# Read the file krdu-2001-2010.csv which is comma delimited
# Store the data in data
data = np.loadtxt('krdu-2001-2010.csv',delimiter = ',')

In [8]:
check('1: data', data, points=8)

1: data appears correct


## Compute the day of the week

Then work out the day of the week for each date (0 is Monday)

In [9]:
# leave your result in dayOfWeek which will be a
# NumPy array that has as many rows as data but
# the datetime.weekday() of the date in the corresponding
# row of data's first three columns (column with index 0, 1, 2)

dayOfWeek = np.zeros(len(data), dtype=int)
for day in range(len(data)): # Walk through each year 2000 through 2100
    year = int(data[day][0])
    month = int(data[day][1])
    day1 = int(data[day][2])
    dayOfWeek[day] = (date(year,month,day1).weekday())
dayOfWeek = np.array(dayOfWeek)
#print(dayOfWeek)

print('The length of dayOfWeek is', len(dayOfWeek), 'which should equal the length of data[:, 0]', len(data))

ValueError: year 0 is out of range

In [ ]:
check('2: days of week', dayOfWeek, points=20)

## Compute rainfall amounts

Write a function that takes two arguments, an array of day numbers and
an array of rainfall amounts and returns the midweek and weekend rain as
defined by the NASA researchers. Remember you can return multiple results
by packing them into a tuple.

I **strongly recommend** using numpy array boolean functions to group your days
into midweek and weekend; looping is likely to be too slow when you
call your function thousands of times in the next part.
You should not write a for loop that tests if each item!



In [ ]:
# Put the code for your function here

def rainfallamts (days,rain):
    midwk = []
    wkend = []
    for day in range(len(days)):
        if days[day] > 4 or days[day] == 0:    
            wkend.append(rain[day])
        if days[day] > 0 and days[day] < 4:
            midwk.append(rain[day]) 
    # now call your function and store the returned values in variables midwk
    # and wkend
    
    return np.average(midwk), np.average(wkend)

# then compute their difference in variable delta

midwk, wkend = rainfallamts(dayOfWeek,data[:,3])                     


delta = midwk - wkend 

print('It rained', midwk, 'inches mid week')
print('It rained', wkend, 'inches weekend')
print('Difference was', delta, 'inches')

In [ ]:
check('3: mid wk rain', midwk, points=15)
check('4: wk end rain', wkend, points=15)
check('5: difference', delta, points=15)

## Test the null hypothesis

Now that you can determine the weekday rain amount and weekend rain amount, we can
finally test the null hypothesis that the day labeling is random
by running your function many times with different permutations of the days
and counting how many times the difference between the means is greater than the delta
you computed above.

In [ ]:
N = 1000 # number of trials, you can experiment with this
count = 0 # number of permutations that produced greater difference than delta

p = 0.0 # your code should compute the probability in this variable

# let's fix the starting point for random numbers to try to reduce the
# variability from run to run. Of course, in a real simulation you wouldn't
# do this but it should make grading easier.

np.random.seed(0)

# Write your code here to loop N times.   For each loop randomize the dayOfWeek,
# use data[:, 3] with the shuffled day of the week and get the weekday and weekend
# rain amounts similar to the previous cell.


for indx in range(N):
    random_day_mean_midwk, random_day_mean_wknd = rainfallamts((np.random.permutation(dayOfWeek)), data[:,3])
    random_day_diff = random_day_mean_midwk - random_day_mean_wknd
    if random_day_diff > delta:
        count += 1
        
p = count / N 

print('p =', p)

In [ ]:
# Note: because of the randomness your answer may be quite different
# from the "expected" value below and still be correct. I have set
# the tolerance pretty wide.

check('6: p value', p, points=25, precision=1)

## Results

100,000 reps gives me a p-value of 0.06 with about 55 seconds of computation. Not quite significant by the common standard of 0.05. As I said earlier, we don't have enough data to test the hypothesis; the NASA guys used the entire southeast. 

## Done!
<img src="restartAndClearOutput.png" width="300" style="float: right" />

Now go back, restart the kernel (menu <font color="green">Kernel</font>-><font color="green">Restart and Clear</font>), and then Shift-Enter your way through the notebook to run all the cells again so you an be sure all your code will work as you expect during grading.

## Saving your work
<img src="saveAndCheckpoint.png" width="300" style="float: right" />

Now save your work by going to (menu <font color='green'>File</font>-><font color='green'>Save and Checkpoint</font>)

In [ ]:
report(Author, Collaborators)
print('The submit button for the assignment is in the unlocker notebook')

## Close this notebook and then go back to the AssignmentUnlocker
<img src="closeAndHalt.png" width="300" style="float: right" />

Before going back to the AssignmentUnlocker and submit your work, you'll need to go close this
notebook (menu <font color='green'>File</font>-><font color='green'>Close and Halt</font>.)


Note that if you actually saved your work you should not see the leaving site message below.
If you do see the `Leave Site` warning, cancel and save your work again.
<br />
<img src="leaveSite.png" width="300" style="float: left" />